In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!nvidia-smi

Mon Jul 15 08:23:02 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P0             25W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!git clone https://github.com/lamnguyenvu98/YOWOv2-PL.git

In [ ]:
%cd YOWOv2-PL

/kaggle/working/YOWOv2-PL
/kaggle/working/YOWOv2-PL


In [ ]:
!mkdir /kaggle/working/lightning_logs

In [ ]:
!git pull

In [ ]:
%%capture
!pip install .

In [ ]:
import wandb
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

my_secret = user_secrets.get_secret("wandb_api_key") 

wandb.login(key=my_secret)

In [12]:
# !rm /kaggle/working/logs/config.yaml

In [ ]:
!yowo fit -c configs/yowov2_nano_ucf24_jhmdb21.yaml \
    --seed_everything=42 \
    --data.data_dir="/kaggle/input/ucf24-spatial-temporal-localization-yowo/ucf24" \
    --data.batch_size=64 \
    --data.num_workers="auto" \
    --model.scheduler_config.scheduler.milestones="[2, 4, 5]" \
    --model.scheduler_config.scheduler.gamma=0.5 \
    --model.scheduler_config.interval="epoch" \
    --model.loss_config.focal_loss="true" \
    --model.warmup_config.scheduler.max_iter=250 \
    --model.warmup_config.interval="step" \
    --model.metric_max_detection_thresholds='[1, 10, 100]' \
    --trainer.max_epochs=7 \
    --trainer.accumulate_grad_batches=16 \
    --trainer.accelerator="gpu" \
    --trainer.devices=1 \
    --trainer.precision="16-mixed" \
    --trainer.strategy="auto" \
    --trainer.callbacks+="lightning.pytorch.callbacks.ModelCheckpoint" \
    --trainer.callbacks.monitor="map" \
    --trainer.callbacks.filename="{epoch}-{map:.4f}" \
    --trainer.callbacks.mode="max" \
    --trainer.callbacks.save_last="true" \
    --trainer.callbacks.save_top_k=3 \
    --trainer.logger.class_path="lightning.pytorch.loggers.WandbLogger" \
    --trainer.logger.name="experiment-yowo-nano_2024-07-15_18:15" \
    --trainer.logger.project="spatial-temporal-action-detection-yowov2" \
    --trainer.logger.tags="['P100', 'ucf24']" \
    --trainer.logger.log_model="all" \
    --trainer.default_root_dir="/kaggle/working/lightning_logs" \
#     --trainer.logger.id="wnaoj0mf"
#     --trainer.callbacks.dirpath="/kaggle/working/checkpoints" \
#     --trainer.logger.save_dir="/kaggle/working/logs" \
#     --trainer.callbacks.every_n_train_steps=500 \
#     --ckpt_path="/kaggle/working/checkpoints/last-v1.ckpt" \
#     --ckpt_path="/kaggle/working/checkpoints/last.ckpt" \
#     --trainer.logger.entity="nguyen" \
#     --trainer.logger.class_path="lightning.pytorch.loggers.CSVLogger" \ # add logger
#     --trainer.logger.init_args.save_dir="/kaggle/working/logs" \
#     --trainer.logger.init_args.name="yowov2_free_nano" \
#     --ckpt_path="/kaggle/working/work_dir/last.ckpt" \
#     --trainer.fast_dev_run="true" \

In [15]:
# Validation
! yowo validate -c configs/yowov2_nano_ucf24_jhmdb21.yaml \
    --data.batch_size=32 \
    --data.data_dir="/kaggle/input/ucf24-spatial-temporal-localization-yowo/ucf24" \
    --data.num_workers="auto" \
    --model.scheduler_config.scheduler.milestones="[2, 4, 5]" \
    --model.scheduler_config.scheduler.gamma=0.5 \
    --model.scheduler_config.interval="epoch" \
    --model.loss_config.focal_loss="true" \
    --model.warmup_config.scheduler.max_iter=250 \
    --model.warmup_config.interval="step" \
    --model.metric_max_detection_thresholds='[1, 10, 100]' \
    --trainer.accumulate_grad_batches=16 \
    --trainer.accelerator="gpu" \
    --trainer.devices=2 \
    --trainer.precision="16-mixed" \
    --trainer.strategy="ddp"

/opt/conda/lib/python3.10/site-packages/lightning/fabric/utilities/seed.py:40: No seed found, seed set to 0
INFO: Seed set to 0
model is  shufflenet_v2_x1_0
init weights...
FPN: pafpn_elan
Head: Decoupled Head
Head: Decoupled Head
Head: Decoupled Head
Loading 2D backbone pretrained weight: YOLO_FREE_NANO
Loading pretrained weight ...
Loading 3D backbone pretrained weight: SHUFFLENET_V2_X1_0
conv_last.0.weight
conv_last.1.weight
conv_last.1.bias
conv_last.1.running_mean
conv_last.1.running_var
conv_last.1.num_batches_tracked
classifier.1.weight
classifier.1.bias
Head: Decoupled Head
Head: Decoupled Head
Head: Decoupled Head
Freeze 2D Backbone ...
Freeze 3D Backbone ...
INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO: [rank: 0] Seed set to 0
INFO: Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2
INFO: [rank: 1] Seed set to 0
model is  shufflene

In [19]:
!ls -l /kaggle/working/lightning_logs

total 81328
-rw-r--r-- 1 root root 20817881 Jul 12 14:03 'epoch=0-total_loss=5.0536.ckpt'
-rw-r--r-- 1 root root 20819673 Jul 12 16:59 'epoch=3-total_loss=2.9880.ckpt'
-rw-r--r-- 1 root root 20819673 Jul 12 22:23  last-v1.ckpt
-rw-r--r-- 1 root root 20819673 Jul 12 14:03  last.ckpt


In [8]:
!mv /kaggle/working/checkpoints/last-v1.ckpt /kaggle/working/checkpoints/last.ckpt

In [15]:
import torch
import os
path = "/kaggle/working/checkpoints"
ckp = torch.load(os.path.join(path, "last-v1.ckpt"), map_location=torch.device("cpu"))
ckp['epoch'], ckp['global_step']

(5, 13500)

In [21]:
!cp /kaggle/working/checkpoints/last-v1.ckpt /kaggle/working/

In [20]:
from IPython.display import FileLink
FileLink('last-v1.ckpt')

/kaggle/working/checkpoints/last-v1.ckpt